In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130811 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty

In [0]:
import os
os.chdir('drive/Competitions/News-to-Predict-Stock-Movements')

In [6]:
!pip install glove_python

     |████████████████████████████████| 266kB 6.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


# Environment setting

In [0]:
import codecs
import json

from IPython.display import display
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib as mpl

import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import time

import ast
import jieba
from gensim.models import word2vec
from gensim.models import FastText
from glove import Glove, Corpus

warnings.filterwarnings("ignore")
# %matplotlib

stopwords = [line.strip() for line in codecs.open(
    r'./Code/stopwords.txt', 'r', 'utf-8').readlines()]

In [0]:
text = pd.read_csv(r'./Data/TRAINSET_NEWS.csv')
stock = pd.read_csv(r'./Data/TRAINSET_STOCK.csv')

# Prepare target vector

In [0]:
stock.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1
1,801010,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1
2,801010,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0
3,801010,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1
4,801010,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1


In [9]:
target_df = (stock.loc[(stock['y'] == 1)]).groupby(['trade_date', 'ts_code'])[
    'y'].count().unstack().fillna(0).astype(int).reset_index()
target_df['trade_date'] = pd.to_datetime(
    target_df['trade_date'], format='%Y%m%d', errors='ignore')
target_df.head()

ts_code,trade_date,801010,801020,801030,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600
0,2014-04-01,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,2014-04-02,1,1,0,1,1,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,0,0
2,2014-04-03,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,1
3,2014-04-04,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2014-04-08,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# Load word embedding

## Load word2vec

In [0]:
w2vmodel = word2vec.Word2Vec.load("./Code/w2v.model")

## Load GloVe

In [0]:
corpus_model = Corpus.load('./Code/glov_corpus.model')
glove = Glove.load('./Code/glove.model')

## Load Fasttext

In [0]:
fttmodel = FastText.load(r'./Code/fasttext.model')

## Concatenate vector

In [15]:
glove.word_vectors.shape, len(fttmodel.wv.vocab), len(w2vmodel.wv.vocab)

((126565, 100), 126565, 126565)

In [16]:
# word_list = list(w2vmodel.wv.vocab.keys())
vector_size = 100
word_index = glove.dictionary
nb_words = len(word_index)

count = 0.5
embedding_word2vec_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_vector = w2vmodel[word] if word in w2vmodel else None
    if embedding_vector is not None:
        count += 1
        embedding_word2vec_matrix[i] = embedding_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_word2vec_matrix[i] = unk_vec


glove_count = 0
embedding_glove_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_glove_vector = glove.word_vectors[glove.dictionary[word]
                                                ] if word in glove.dictionary else None
    if embedding_glove_vector is not None:
        glove_count += 1
        embedding_glove_matrix[i] = embedding_glove_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_glove_matrix[i] = unk_vec


ftt_count = 0
embedding_ftt_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_ftt_vector = fttmodel[word] if word in fttmodel else None
    if embedding_ftt_vector is not None:
        ftt_count += 1
        embedding_ftt_matrix[i] = embedding_ftt_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_ftt_matrix[i] = unk_vec

embedding_matrix = np.concatenate(
    (embedding_word2vec_matrix, embedding_glove_matrix, embedding_ftt_matrix), axis=1)
print(embedding_matrix.shape, count * 1.0 /
      embedding_matrix.shape[0], glove_count*1.0/embedding_matrix.shape[0], ftt_count*1.0/embedding_matrix.shape[0])

100%|██████████| 126565/126565 [00:01<00:00, 74287.18it/s]


(126566, 300) 0.9999960494919646 0.9999920989839294 0.9999920989839294


# Load word segment

In [21]:
seg_text = pd.read_csv(r'./Code/seg_words.csv')
seg_text['date'] = pd.to_datetime(
    seg_text['date'], format='%Y%m%d', errors='ignore')
seg_text.head()

,date,title,content
0,2014-04-14,"['习近平', '空军', '机关', '调研', '时', '强调', '加快', '建设...","['中共中央', '总书记', '国家', '主席', '中央军委', '主席', '习近平..."
1,2014-04-14,"['利比亚', '临时政府', '总理', '辞职']","['本月', '8', '号', '刚刚', '正式', '任命', '利比亚', '临时政..."
2,2014-04-14,"['关注', '乌克兰', '局势']","['代行', '乌克兰', '总统', '职责', '乌克兰', '议长', '图尔', '..."
3,2014-04-14,"['国内', '联播', '快讯']","['低碳', '中国', '行', '正式', '启动', '国家', '发展', '改革'..."
4,2014-04-14,"['刘汉', '36', '人涉', '黑案', '继续', '开庭审理']","['刘汉', '刘维', '36', '人', '涉嫌', '犯', '组织', '领导',..."


In [0]:
def merge_func(x):
    x = ast.literal_eval(x)
    r_list = []
    for i in x:
        r_list += i
    return r_list

## Merge news title from the same date

In [0]:
join_title = seg_text[['date', 'title']]
join_title = join_title.groupby(
    ['date'])['title'].apply(','.join).reset_index()
join_title['title'] = join_title['title'].apply(merge_func)

## Merge news content from the same date

In [0]:
join_content = seg_text[['date', 'content']]
join_content = join_content.groupby(
    ['date'])['content'].apply(','.join).reset_index()
join_content['content'] = join_content['content'].apply(merge_func)

# Contact title and content

In [25]:
joined_text = join_title.merge(join_content, on='date')
joined_text.head()

,date,title,content
0,2014-04-14,"[习近平, 空军, 机关, 调研, 时, 强调, 加快, 建设, 一支, 空天, 一体, 攻...","[中共中央, 总书记, 国家, 主席, 中央军委, 主席, 习近平, 14, 日, 专程到,..."
1,2014-04-15,"[医生, 贾永青, 传递, 爱, 感动, 国际, 联播, 快讯, 搜寻, MH370, 航班...","[几天, 我台, 走, 基层, 节目, 连续, 报道, 河北, 定州, 人民, 医院, 32..."
2,2014-04-16,"[国际, 联播, 快讯, 关注, 乌克兰, 局势, 乌, 军队, 东部, 地区, 开展, 强...","[约旦, 驻, 利比亚, 大使, 遭绑架, 利比亚, 外交部, 15, 号, 证实, 约旦,..."
3,2014-04-17,"[凡人, 善举, 广西, 市民, 见义勇为, 巧施, 妙计, 擒, 劫匪, 国际, 联播, ...","[前两天, 广西北海, 一位, 市民, 目睹, 一起, 抢夺案, 后, 没有, 选择, 离开..."
4,2014-04-18,"[国际, 联播, 快讯, 俄罗斯, 总统, 俄, 民众, 直接对话, 普京, 乌, 境内, ...","[伊朗, 举行, 建军节, 阅兵式, 18, 号, 伊朗, 首都, 德黑兰, 南郊, 霍梅尼..."


# Padding size

In [0]:
title_size = int(np.percentile(joined_text['title'].str.len(), 95))
content_size = int(np.percentile(joined_text['content'].str.len(), 95))

# Replace word embedding

In [27]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


# News 2 Sequence

In [0]:
token = Tokenizer()
token.word_index = word_index

In [29]:
title_sequences = token.texts_to_sequences(join_title['title'].values)
title_vec = pad_sequences(title_sequences, maxlen=title_size,
                          padding='post', truncating='post', value=nb_words)

content_sequences = token.texts_to_sequences(join_content['content'].values)
content_vec = pad_sequences(content_sequences, maxlen=content_size,
                            padding='post', truncating='post', value=nb_words)
title_vec.shape, content_vec.shape

((1814, 160), (1814, 3305))

# Rolling

In [0]:
train_roll_window = 10
target_roll_window = 5

## Target rolling

In [0]:
tmp_target = target_df.iloc[::-1]
limit_target_index = (tmp_target['trade_date'] >= '2014-04-25')
tmp_target = tmp_target[limit_target_index]

In [32]:
tmp_target.shape

(1049, 35)

In [33]:
rolling_date = []
target_matrix = np.zeros(
    (tmp_target.shape[0]-target_roll_window+1, 34*target_roll_window))
for i in tqdm(range(0, tmp_target.shape[0]-4)):
    tmp_i = tmp_target.iloc[i:i+5]
    rolling_date.append(list(tmp_i['trade_date']))
    target_matrix[i] = (tmp_i.drop(
        columns=['trade_date'], axis=1).values).reshape(1, -1)[0]

100%|██████████| 1045/1045 [00:01<00:00, 743.78it/s]


In [34]:
target_matrix.shape

(1045, 170)

## Data rolling

In [35]:
train_title_matrix = np.zeros(
    (target_matrix.shape[0], title_size*train_roll_window*target_roll_window))
train_content_matrix = np.zeros(
    (target_matrix.shape[0], content_size*train_roll_window*target_roll_window))
train_title_matrix.shape, train_content_matrix.shape

((1045, 8000), (1045, 165250))

In [36]:
for i in tqdm(range(0, len(rolling_date))):
    tmp_title_row = np.zeros(
        (target_roll_window, title_size*train_roll_window))
    tmp_content_row = np.zeros(
        (target_roll_window, content_size*train_roll_window))

    for j in range(0, len(rolling_date[i])):
        index_range = (join_title['date'] >= (
            rolling_date[i][j] - pd.DateOffset(train_roll_window))) & (join_title['date'] < rolling_date[i][j])
        tmp_title_row[j] = title_vec[index_range].reshape(1, -1)[0]
        tmp_content_row[j] = content_vec[index_range].reshape(
            1, -1)[0]
    train_title_matrix[i] = tmp_title_row.reshape(1, -1)[0]
    train_content_matrix[i] = tmp_content_row.reshape(1, -1)[0]

100%|██████████| 1045/1045 [00:08<00:00, 128.52it/s]


In [37]:
train_title_matrix.shape, train_content_matrix.shape

((1045, 8000), (1045, 165250))

# Nerual network

In [0]:
import keras
from keras.utils import np_utils
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *

## BiGRU

In [0]:
def bi_gru_model(sent_length, embeddings_weight, output_num):
    print("get_text_gru3")
    content = Input(shape=(sent_length,), dtype='int32')
    embedding = Embedding(
        name="word_embedding",
        input_dim=embeddings_weight.shape[0],
        weights=[embeddings_weight],
        output_dim=embeddings_weight.shape[1],
        trainable=False)

    x = SpatialDropout1D(0.2)(embedding(content))

    x = Bidirectional(CuDNNGRU(200, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(200, return_sequences=True))(x)

#     x = Bidirectional(GRU(200, return_sequences=True))(x)
#     x = Bidirectional(GRU(200, return_sequences=True))(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)

    conc = concatenate([avg_pool, max_pool])

    x = Dropout(0.2)(Activation(activation="relu")(
        BatchNormalization()(Dense(1000)(conc))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    output = Dense(output_num, activation="softmax")(x)

    model = Model(inputs=content, outputs=output)
    model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [40]:
title_gru_model = bi_gru_model(sent_length=(title_size*train_roll_window*target_roll_window),
                         embeddings_weight=embedding_matrix, output_num=34*target_roll_window)

get_text_gru3
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8000)         0                                            
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 8000, 300)    37969800    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 8000, 300)    0           word_embedding[0][0]             
__________________________________________________________________________________________________
bidirec

In [41]:
title_gru_model.fit(train_title_matrix, target_matrix)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
1045/1045 [==============================] - 176s 169ms/step - loss: 530.3747 - acc: 0.0057


In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import  accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

os.environ["CUDA_VISIBLE_DEVICES"] ="0"

from keras.utils.training_utils import multi_gpu_model
from keras import backend as K 

In [0]:
kf = KFold(n_splits=5, shuffle=True, random_state=520).split(train_title_matrix)

In [0]:
for i, (train_fold, test_fold) in enumerate(kf):
        X_train, X_valid, = train_title_matrix[train_fold, :], train_title_matrix[test_fold, :]
        y_train, y_valid = target_matrix[train_fold], target_matrix[test_fold]

        print(i, 'fold')
        
        the_path = './'

        title_gru_model = bi_gru_model(sent_length=(title_size*train_roll_window*target_roll_window),
                         embeddings_weight=embedding_matrix, output_num=34*target_roll_window)
        
        early_stopping = EarlyStopping(monitor='val_acc', patience=6)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=3)
        checkpoint = ModelCheckpoint(the_path + str(i) + '.hdf5', monitor='val_acc', verbose=2, save_best_only=True, mode='max',save_weights_only=True)
        if not os.path.exists(the_path + str(i) + '.hdf5'):
            print("error")
            title_gru_model.fit(X_train, y_train,
                      epochs=100,
                      batch_size=16,
                      validation_data=(X_valid, y_valid),
                      callbacks=[early_stopping, plateau, checkpoint],
                      verbose=2)


        title_gru_model.load_weights(the_path + str(i) + '.hdf5')

        print (": valid's accuracy: %s" % f1_score(lb.inverse_transform(np.argmax(y_valid, 1)), 
                                                          lb.inverse_transform(np.argmax(title_gru_model.predict(X_valid), 1)).reshape(-1,1),
                                                          average='micro'))
    
        train_model_pred[test_fold, :] =  title_gru_model.predict(X_valid)
        test_model_pred += title_gru_model.predict(test_)
        
        del title_gru_model; gc.collect()
        K.clear_session()

0 fold
get_text_gru3
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 8000)         0                                            
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 8000, 300)    37969800    input_6[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_6 (SpatialDro (None, 8000, 300)    0           word_embedding[0][0]             
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 8000, 400)    602400      spatial_dropout1d_6[0][0]        
________________________________________________________________________________________